### PbSnO Energy Band simulation example
First, load all the needed packages and create the system putting all parameters together

In [ ]:
import torch
import torch.nn.functional as F
from NNTB.crystal import Crystal
from NNTB.system import TBSystem
import numpy as np
from e3nn import o3, nn
from e3nn.o3 import Irreps, spherical_harmonics
from NNTB.balanced_irreps import BalancedIrreps, WeightBalancedIrreps
from torch_scatter import scatter
import os
from torch_geometric.data import Data, Batch
import matplotlib.pyplot as plt
from NNTB.hamiltonian_irrep import HamiltonianIrrepTransformer
from NNTB.balanced_irreps import BalancedIrreps

# We import the warnings library to avoid the warnings from e3nn in the new versions of python...
import warnings
warnings.filterwarnings('ignore') 

torch.set_default_dtype(torch.float32)
torch.manual_seed(42)

# The diccionaries with the element information are created
element_encoding = {'O': 0, 'Pb': 1, 'Sn': 2}

# We define the orbitals for each element
NNTB_orbitals = "1s1p"

#From numerical orbitals used for DFT
abacus_orbitals = { 
    'Pb': "2s2p2d1f",
    'Sn': "2s2p2d1f",
    'O': "2s2p1d"
    }

# Folder with all the atom information
dataFolder = "./"    # for example "./" or "../DataFolder"

# We need to indicate the representations for the inner layers
hidden_irreps = BalancedIrreps(4, 128, False)
#You can use custom irreps too: e.g. hidden_irreps = Irreps("38x0e+38x0o+4x1e+4x1o+2x2e+2x2o+2x3e+2x3o+1x4e+1x4o")

calculator = TBSystem(element_encoding,
                  orbitalstrings=NNTB_orbitals, #Orbitals for the NNTB
                  orbitalabacus=abacus_orbitals,   #Orbitals from the ABACUS LCAO orbitals
                  spin=False,           #Spin Hamiltonian?
                  neighbour_cutoff=8.0, #neighbour cutoff in Angstrom
                  gauss_width=500.0,   #Parameter for distance encoder (gaussian width)
                  dist_onehotsize=128, #Distance encoder size
                  max_atoms=10,         #Maximum number of atoms in the system, it can be bigger than element_encoding to  allow later training
                  neighbour_cells=2,    # Number of neighbour cells to consider for periodic boundary conditions
                  model = 'edgesegnn',  # The model to be used
                  hidden_irreps = hidden_irreps, # The representations for the inner layers, can be None then it will be calculated using lmax_irrep_hidden and hidden_features
                  lmax_irrep_Y = 4,        # Maximum l for the spherical harmonics for node and edge attributes
                  lmax_irrep_hidden = 4,  # Maximum l for the hidden layers (ignored if hidden_irreps is not None)
                  hidden_features = 128,  # Number of features for the hidden layers (ignored if hidden_irreps is not None)
                  convolution_layers = 3, # Number of convolutional layers in the message passing step
                  MLP_layers = 2,         # Number of layers for the MLPs for tensor products depending on the distance (weightNet)
                  weight_hidden= 64,        # Number of features for the hidden layers in the weightNet
                  weightNetType='MLP', # Type of weightNet, can be MLP or KAN
                  norm = "batch")      #Normalization for the output of the hidden layers, can be None, batch or instance
#check other parameters for the calculator in the documentation...


# Other environment configurations..

#printing options
torch.set_printoptions(precision=3, linewidth=200, sci_mode=False)
np.set_printoptions(precision=3, linewidth=200, edgeitems=10, suppress=True, threshold=10000)

#Number of threads, adjust to the number of cores in your CPU
torch.set_num_threads(8)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device='cpu'

print(f"Device:{device}")
print(f"pyTorch Version:{torch.__version__}")


# Plotting in the notebook, IPhython?
%matplotlib inline

Construct the the NN model (This step takes approx 1 min.)
* After constructing, you can save the original model with the random parameters before training
* If a previous model was saved after training, you can load it (this import all the trained parameters and replace the default random parameters)

In [ ]:
model = calculator.gettorchModel("Hamiltonian")
#torch.save(model.state_dict(), f'{dataFolder}/Models/modelPbSnO-02-07-c.pth')
#model.load_state_dict(torch.load(f'{dataFolder}/Models/modelPbSnO-02-07-a.pth'))

Some models require to precompute parameters (otherwise is too slow to calculate on each epoch)
* e.g. Irreducible representations and spherical harmonics of vectors are calculated here.

You can process a whole folder. The format is a {prefix}.cif followed by DFT data: .log, .HR, .SR files (HR_SO and SR_SO if includes spin orbit)

In [ ]:
calculator.processGraphFolder(f'{dataFolder}/PbSnOtest/',f'{dataFolder}/PbSnO_testprocessed/')
calculator.processGraphFolder(f'{dataFolder}/PbSnOtrain/',f'{dataFolder}/PbSnO_trainprocessed/')

Now, you can train the model. You can follow the training evolution in the "./runs" directory opening the tensorboard extension

In [ ]:
from NNTB.trainwithband import trainModel
trainModel(calculator,model,
            datafolder=f'{dataFolder}/PbSnO_trainprocessed',        #Folder with the processed data
            validation_folder=f'{dataFolder}/PbSnO_testprocessed',  #Folder with the processed data for validation
            runname='Example',                            #Name for the run
            learn_rate=0.005,weight_decay=8.0e-8,batchsize=1,       #Learning rate, weight decay and batch size
            device='cuda',epochs=6000,          #Device to use, number of epochs
            SymmGauge=True,EnergyGauge=False,   #Symmetry and energy gauge
            scheduler_patience = 30, scheduler_factor = 0.9,  #Scheduler parameters
            bandloss = True,                    #use Band loss in the training
            bandloss_beta = 0.005, bandloss_patience = 3000, bandloss_max_lr = 0.002, #For stability, requires initial training before use bandloss. total loss = bandloss + beta * loss, iterations to start bandloss, max learning rate for bandloss and max learning rate
            adaptive_patience = -1,            #Iterations to start adaptive distance encoder, -1 for never use it
            modeltype='Hamiltonian')           #Type of model, Hamiltonian (with or without SOC) or Overlap

You can save the model parameters after training

In [5]:
torch.save(model.state_dict(), './models/modelPbSnO-date.pth')

A way of test it is to compare with a reference data

In [4]:
from NNTB.hamiltonian_irrep import HamiltonianIrrepTransformer
#Load reference model
crystal_test = Crystal(f"{dataFolder}/structures/", "PbO_alpha", calculator, undirected=True)
data_test = crystal_test.graph
calculator.processGraph(data_test)
data_test.to('cuda')  #Move to GPU as the model is there

#Evaluate the model:
out_test = calculator.hamiltoniantransformer.from_irrep_Hamiltonian(model(data_test).to('cpu'))

Helper function. Add a plot function that overlaps the matrix values inside the plot

In [5]:
def plot_matrix_values(matrix):
    plt.imshow(matrix, cmap='viridis', interpolation='nearest')

    # Obtener los límites de los valores para definir el color dinámico
    min_val, max_val = matrix.min(), matrix.max()

    # Añadir los valores de cada elemento sobre la imagen
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            value = matrix[i, j]
            # Definir el color dinámico: blanco para valores bajos, negro para altos
            color = 'white' if value < (min_val + max_val) / 2 else 'black'
            plt.text(j, i, f'{value:.5f}', ha='center', va='center', color=color)

    plt.colorbar()  # Mostrar la barra de color
    plt.show()

Compare the mean-absolute error(MAE) for ON-SITE energy:

In [ ]:
%matplotlib inline
matrix1 = np.zeros((4,4))
n_eval = 0
for i in range(crystal_test.graph.hopping.size(0)):
    if(crystal_test.graph.selfenergy[i].item() == True):
        n_eval += 1
        matrix1 = matrix1 + (crystal_test.graph.hopping[i].cpu().numpy()-out_test[i].cpu().detach().numpy())**2
    
matrix1 = np.sqrt(matrix1 / n_eval)
plot_matrix_values(matrix1)

Compare the mean-absolute error(MAE) for OFF-SITE energy:

In [ ]:
%matplotlib inline
matrix1 = np.zeros((4,4))
n_eval = 0
for i in range(crystal_test.graph.hopping.size(0)):
    if(crystal_test.graph.selfenergy[i].item() == False):
        n_eval += 1
        matrix1 = matrix1 + (crystal_test.graph.hopping[i].cpu().numpy()-out_test[i].cpu().detach().numpy())**2
matrix1 = np.sqrt(matrix1 / n_eval)
plot_matrix_values(matrix1)

Compare expected vs modeled results for each hamiltoniam bloch (orbital s x s, s x p, p x p)

In [ ]:
import matplotlib.pyplot as plt

# Assuming pred_tensor and target_tensor are your tensors of shape [n, 4, 4]
n = data_test.hopping.size(0)

%matplotlib inline
# Plotting
plt.figure(figsize=(8, 6))
plt.scatter(data_test.hopping.cpu().detach()[:,0,0], out_test.cpu().detach()[:,0,0],c='black', label='s-s orbital block')
plt.scatter(data_test.hopping.cpu().detach()[:,0,1:], out_test.cpu().detach()[:,0,1:],c='red', label='s-p orbital block')
plt.scatter(data_test.hopping.cpu().detach()[:,1:,1:], out_test.cpu().detach()[:,1:,1:],c='green', label='p-p orbital block')

plt.xlabel('Target')
plt.ylabel('Predicted')
plt.title('Scatter Plot of Predicted vs Target Values')
plt.legend()
plt.grid(True)
plt.show()

Finally, compare the bandenergy !!!

In [9]:
from NNTB.tightbinding import bandfromgraph_torch

In [ ]:
kpoints = np.load(f'{dataFolder}/kpoints.npy')
band = bandfromgraph_torch(data_test.cpu(),calculator,torch.tensor(kpoints,dtype=torch.float32))
data_test.hopping=(out_test.cpu().detach())
band2 = bandfromgraph_torch(data_test.cpu().detach(),calculator,torch.tensor(kpoints,dtype=torch.float32))

import matplotlib.pyplot as plt

x = range(0,len(band2))

for i,values in enumerate(band2):
    plt.scatter(np.full(len(values),i), values, c="black", s=2.0)

for i,values in enumerate(band):
    plt.scatter(np.full(len(values),i), values, c="red", s=2.0)

# Add labels and legend
plt.xlabel('x')
plt.ylabel('y')
plt.ylim(-10,20)

# Show the plot
plt.show()

### Extra Tests
Compare with the superlattice

In [11]:
from NNTB.hamiltonian_irrep import HamiltonianIrrepTransformer
crystal_test = Crystal(f"{dataFolder}/structures/", "2PbO_alpha-2SnO_alpha", calculator, undirected=True)
data_test = crystal_test.graph
calculator.processGraph(data_test)
data_test.to('cuda')
out_test = calculator.hamiltoniantransformer.from_irrep_Hamiltonian(model(data_test).to('cpu'))

In [ ]:
kpoints = np.load(f'{dataFolder}/kpoints.npy')
band = bandfromgraph_torch(data_test.cpu(),calculator,torch.tensor(kpoints,dtype=torch.float32))
data_test.hopping=(out_test.cpu().detach())
band2 = bandfromgraph_torch(data_test.cpu().detach(),calculator,torch.tensor(kpoints,dtype=torch.float32))

import matplotlib.pyplot as plt

x = range(0,len(band2))

for i,values in enumerate(band2):
    plt.scatter(np.full(len(values),i), values, c="black", s=2.0)

for i,values in enumerate(band):
    plt.scatter(np.full(len(values),i), values, c="red", s=2.0)

# Add labels and legend
plt.xlabel('x')
plt.ylabel('y')
plt.ylim(-10,20)

# Show the plot
plt.show()